In [1]:
pip install opencv-python


In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

# Use the first frame as the base
ret, first_frame = cap.read()
first_frame = cv2.resize(first_frame, (640, 480))
gray_base = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
gray_base = cv2.GaussianBlur(gray_base, (21, 21), 0)

# Motion buffer for slow movement
motion_accumulator = np.zeros_like(gray_base, dtype=np.float32)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (640, 480))
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)

    # Frame difference
    delta = cv2.absdiff(gray_base, gray)
    _, thresh = cv2.threshold(delta, 10, 255, cv2.THRESH_BINARY)  # Lowered from 25 to 10
    thresh = cv2.dilate(thresh, None, iterations=2)

    # Accumulate small differences over time
    cv2.accumulateWeighted(thresh, motion_accumulator, 0.1)
    motion_mask = cv2.convertScaleAbs(motion_accumulator)

    # Find contours on the motion mask
    contours, _ = cv2.findContours(motion_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    motion_detected = False

    for contour in contours:
        if cv2.contourArea(contour) < 500:  # Lowered from 1000 to 500
            continue
        motion_detected = True
        (x, y, w, h) = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    if motion_detected:
        cv2.putText(frame, "Motion Detected", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("Motion Tracking - Slow Motion Friendly", frame)

    gray_base = gray.copy()  # Update background

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
